# * *PRÁCTICA FINAL SOTO ANÁLISIS DE SENTIMIENTOS* *

In [1]:
import os
import re
import sys
import warnings
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


import spacy
import numpy as np
import pandas as pd

sys.path.append(os.path.realpath('../'))
# from scripts.preprocess_data import Preprocess

warnings.filterwarnings('ignore')

PATH_DATA = "../"
DEV_FILE = "dev/dev.tsv"
TRAIN_FILE = "Corpus/train.tsv"

In [2]:
def basic_processing(X):
    tweets_clean = []
    
    nlp = spacy.load("es_core_news_sm")

    for sen in range(0, len(X)):
        tweet = str(X[sen])
        tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove tweeted at
        #tweet = re.sub('(#\s+[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hashtags at
        tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hashtags + espacio at
        doc = nlp(tweet)

            #obtenemos la frase lematizada 
        lista_tweet=[]
        newSentence=''
        for token in doc:
            if (
                #not token.is_punct
                #and not token.is_emoji
                #and not token.like_num
                not token.is_stop
                and not token.like_url
                and not token.is_space
                and not token.pos_ == "SYM"
                #and not token.pos_ == "DET"
                and not token.pos_ == "X"
                #and not token.pos_ == "SCONJ"
                #and not token.pos_ == "CONJ"
                #and not token.pos_ == "ADP"
                #and not token.pos_ == "INTJ"
                and not token.pos_ == "NUM"
                #and not token.pos_ == "PRON"
                #and not token.pos_ == "PROPN"
            ):
                newSentence = ' '.join([newSentence,token.lemma_.lower()])
                #lista_tweet.append(newSentence)
        tweets_clean.append(newSentence)
    print(tweets_clean)
    return tweets_clean

In [3]:
def processTweetsTSV(corpus_path, trainingFile):
    if (not os.path.exists(corpus_path)):
        os.mkdir(corpus_path)

    df_dev = pd.read_csv(trainingFile, sep="\t", usecols = ['id', 'tweet','emotion'])

    anger_path = corpus_path+'\\ANGER'
    disgust_path = corpus_path+"\\DISGUST"
    fear_path = corpus_path+"\\FEAR"
    joy_path = corpus_path+"\\JOY"
    sadness_path = corpus_path + "\\SADNESS"
    surprise_path = corpus_path + "\\SURPRISE"
    others_path = corpus_path + "\\OTHERS"

    if (not os.path.exists(anger_path)):
            os.makedirs(anger_path)
            os.makedirs(disgust_path)
            os.makedirs(fear_path)
            os.makedirs(joy_path)
            os.makedirs(sadness_path)
            os.makedirs(surprise_path)
            os.makedirs(others_path)

    #print(df_dev)
    df_dev.to_csv("filterDataTrain.csv")

    with open('filterDataTrain.csv', mode='r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file, quotechar='"', delimiter=',',
                                    quoting=csv.QUOTE_ALL, skipinitialspace=True)
        path = ''
        for row in csv_reader:
            #print(row)
            if row["emotion"] == "others":
                path= others_path
            elif row["emotion"] == "anger":
                path= anger_path
            elif row["emotion"] == "disgust":
                path= disgust_path
            elif row["emotion"] == "fear":
                path= fear_path
            elif row["emotion"] == "joy":
                path = joy_path
            elif row["emotion"] == "sadness":
                path = sadness_path
            elif row["emotion"] == "surprise":
                path = surprise_path

            # Creating new file
            f=open(path+"\\"+row['id']+".txt","w", encoding='utf-8')
            f.write(row['tweet'])
            f.close()

In [4]:
def classifier(corpus_path):

    # CorpusTrain and subfolders (categories) must exist
    tweets_data = load_files(corpus_path)
    X, y = tweets_data.data, tweets_data.target

    documents = basic_processing(X)

    vectorizer = CountVectorizer()
    #vectorizer = CountVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1, 1))
    X = vectorizer.fit_transform(documents).toarray()

    tfidfconverter = TfidfTransformer()
    X = tfidfconverter.fit_transform(X).toarray()

    # The data is divided into 20% test set and 80% training set.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    #clf = MultinomialNB().fit(X_train, y_train)
    #clf = BernoulliNB().fit(X_train, y_train)
    #clf = GaussianNB().fit(X_train, y_train)
    #clf = ComplementNB().fit(X_train, y_train)
    #clf = CategoricalNB().fit(X_train, y_train)
    #clf = DecisionTreeClassifier().fit(X_train, y_train)
    #clf = ExtraTreeClassifier().fit(X_train, y_train)
    #clf = LinearSVC().fit(X_train, y_train)
    #clf = SVC().fit(X_train, y_train)
    clf = KNeighborsClassifier().fit(X_train, y_train)
    #y_pred = clf.predict(X_test)

    print(confusion_matrix(y_test,y_pred))
    print("------------------------------------------")
    print(classification_report(y_test,y_pred))
    print("------------------------------------------")
    print("accuracy",accuracy_score(y_test, y_pred))

In [5]:
if __name__ == "__main__":
    trainingFile = "Corpus/train.tsv"
    corpus_path = "CorpusTraining"

    processTweetsTSV(corpus_path, trainingFile)
    # entrenar el clasificador y evaluar el rendimiento
    classifier(corpus_path)

[" b'user chavista ? ! pensar as\\xc3\\xad ir a salir pobre ajajaj .", " b ' mvp episodio aprovechar noche previo a batalla . user", " b'un user enredado .. f\\xc3\\xa1cil rival user", ' b # secretario oea user manifest\\xc3\\xb3 apoyo a adhesi\\xc3\\xb3n militar a constituci\\xc3\\xb3n : " necesario m\\xc3\\xa1s pleno respaldo proceso transici\\xc3\\xb3n democr\\xc3\\xa1tico forma pac\\xc3\\xadfico " user / user', " b'es brutal , cifra tanto p\\xc3\\xa9rdida y moral gente , s\\xc3\\xb3lir francés mundo entero perdido \\xc3\\xadcono grande noche oraci\\xc3\\xb3n ayudar a alma francés semejante acontecimiento est\\xc3\\xa1 luto", " b'y dormir joder", ' b .... y perder madrid ... felicidad ! ! !', " b'no creer m\\xc3\\xa1s messi ! ! ! \\xf0\\x9f\\x91\\x8f\\xf0\\x9f\\x8f\\xbd\\xf0\\x9f\\x91\\x8f\\xf0\\x9f\\x8f\\xbd\\xf0\\x9f\\x91\\x8f\\xf0\\x9f\\x8f\\xbd\\xf0\\x9f\\x91\\x8f\\xf0\\x9f\\x8f\\xbd\\xf0\\x9f\\x91\\x8f\\xf0\\x9f\\x8f\\xbd sos enorme pulgo", " b'el hospital universitario ceuta c

NameError: name 'y_pred' is not defined